In [1]:
import sys
import pandas as pd
import numpy as np
import torch
from torch import Tensor

from matplotlib import pyplot as plt

sys.path.append("../")
from src.log import myLogger
from src.repository.data_repository import DataRepository
from src.checkpoint.checkpoint import Checkpoint

pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 600)

%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
%config InlineBackend.figure_formats = {'png', 'retina'}

/Users/naoya.taguchi/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/Users/naoya.taguchi/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/Users/naoya.taguchi/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/Users/naoya.taguchi/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [2]:
logger = myLogger("../logs/temp.log", exp_id="e000", wdb_prj_id="temp", exp_config=None, use_wdb=False) 
dr = DataRepository(logger=logger, local_root_path="..")

2021-11-08 00:25:54,550 log.py               31   [INFO] [__init__] skip wandb init 


In [7]:
# prep_df = pd.read_csv("../data/preprocessed/train_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_True_False_False_False.csv")
with open("../data/preprocessed/train_BaselineKernelPreprocessorV1_XLMRobertaTokenizerFast_400_True_135_True_False_False_False.pkl", "rb") as fin:
    prep_df = pickle.load(fin)
prep_df

,id,context,question,answer_text,answer_start,language,input_ids,token_type_ids,attention_mask,sequence_ids,offset_mapping,overflowing_batch_id,start_position,end_position,segmentation_position,is_contain_answer_text
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,27,27,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
1,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
3,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",3,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
4,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil,"[0, 69535, 81049, 37368, 153264, 12095, 52989,...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, Non...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",4,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14183,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",0,0,0,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
14184,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",1,351,352,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
14185,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, ...",2,101,102,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
14186,ee569fe45,अमेरिका में जन्मीं अभिनेत्री तथा फैशन डिज़ाइनर...,लीजा रे की पहली फिल्म का नाम क्या था?,नेताजी,2017,hindi,"[0, 40531, 10886, 20322, 471, 47211, 12125, 64...","[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None

In [8]:
prep_df.question.head()

0    மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
1    மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
2    மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
3    மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
4    மனித உடலில் எத்தனை எலும்புகள் உள்ளன?
Name: question, dtype: object

In [9]:
"மனித உடலில் எத்தனை எலும்புகள் உள்ளன?".split()

['மனித', 'உடலில்', 'எத்தனை', 'எலும்புகள்', 'உள்ளன?']

In [ ]:
# "எத்தனை" → how many

In [14]:
from string import punctuation

def calc_duplicated_elems_num_with(offset_mapping, context, question):
    s = 1_000_000_000
    e = 0
    for (os, oe) in offset_mapping:
        if os == -1:
            continue
        s = min(os, s)
        e = min(oe, e)
    context_part = context[s:e]
    duplicated_elems_num = set([context_part_word.strip(punctuation) for context_part_word in context_part.split()]) \
                                                & set(question_word.strip(punctuation) for question_word in question.split())
    return len(duplicated_elems_num)

In [16]:
from tqdm import tqdm

tqdm.pandas()

prep_df["duplicated_elems_num"] = prep_df.progress_apply(lambda row: calc_duplicated_elems_num_with(row["offset_mapping"].tolist(), str(row["context"]), str(row["question"])), axis=1)
prep_df

  0%|          | 1/14188 [00:00<01:28, 159.92it/s]


AttributeError: 'str' object has no attribute 'tolist'

In [10]:
"சத்தோசி நகமோட்டோவால் ".split()

['சத்தோசி', 'நகமோட்டோவால்']

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("../data/dataset/deepset/xlm-roberta-large-squad2/")

In [12]:
dir(tokenizer)

['SPECIAL_TOKENS_ATTRIBUTES',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_tokens',
 '_additional_special_tokens',
 '_batch_encode_plus',
 '_bos_token',
 '_cls_token',
 '_convert_encoding',
 '_convert_id_to_token',
 '_convert_token_to_id_with_added_voc',
 '_create_or_get_repo',
 '_decode',
 '_decode_use_source_tokenizer',
 '_encode_plus',
 '_eos_token',
 '_eventual_warn_about_too_long_sequence',
 '_from_pretrained',
 '_get_padding_truncation_strategies',
 '_get_repo_url_from_name',
 '_mask_token',
 '_pad',
 '_pad_token',
 '_pad_token_type_id',
 '_push_to_hub',
 '_save_pretrained',
 '_sep_token',
 '_tokenizer',
 '_un

In [13]:
tokenizer.tokenize("சத்தோசி நகமோட்டோவால் ")

['▁ச', 'த்த', 'ோ', 'சி', '▁ந', 'க', 'மோ', 'ட்ட', 'ோ', 'வால்', '▁']